In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os
from glob import glob
from ipywidgets import interact
import matplotlib.patches as patches
import pandas as pd
import json
import copy

In [ ]:
"""colab은 out of memory 잘 떠서 권장 X
"""
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
"""zip파일이면 아래 주석 제거
"""
#!unzip '../archive.zip' -d './'

In [ ]:
base_path = '/content/gdrive/MyDrive/Boostcamp_AI_tech/10주차-Competition/kaggle_dataset/data'
# zip파일 풀고 나온 폴더 경로 (data폴더 path)
os.chdir(base_path)
# 현재 working directory를 경로 위치로 변경

# 이미지 및 원본 라벨 확인 (필수 실행)

In [ ]:
annFile = os.path.join(base_path, 'annotations.json')
# 원본 annotation file

coco = COCO(annFile)
classes = coco.loadCats(coco.getCatIds())
classes = [cls['name'] for cls in classes]
df = pd.DataFrame(classes)
print(df)
img_id = coco.getImgIds()
img_info = coco.loadImgs(img_id)

fnames = [info['file_name'] for info in img_info]
ids = [info['id'] for info in img_info]
ann_id = coco.getAnnIds(imgIds=ids)
annotation_info_list = coco.loadAnns(ann_id)

"""
이미지 사이즈가 커서 slide 반응이 좀 느림
"""
@interact(idx=(0, len(annotation_info_list)-1))
def showImg(idx):
    fig, ax = plt.subplots(1, 1, dpi=150)
    img_id = annotation_info_list[idx]['image_id']
    img = io.imread(os.path.join(base_path, fnames[img_id]))
    ax.imshow(img)
    
    x,y,w,h = annotation_info_list[idx]['bbox']
    x,y,w,h = float(x), float(y), float(w), float(h)
    ax.add_patch(
        patches.Rectangle(
            (x,y), w, h,
            edgecolor='white',
            fill=False,
            ),
        )
    text_y = y-30 if y>30 else y+30 
    ax.text(x,text_y, classes[annotation_info_list[idx]['category_id']], color='white', fontsize='5')
    ax.set_xticks([])
    ax.set_yticks([])

# json class id 수정 및 bbox 사이즈 작은 annotation 제거

In [ ]:
tf_dict={0:[25, 31, 58, 59], 
         1:[3, 13, 14, 15, 17, 18, 19, 30, 32, 33, 34, 56], 
         2:[16, 20],
         3:[0, 2, 8, 10, 11, 12, 28, 50, 52],
         4:[6, 9, 23, 26],
         5:[4, 5, 7, 21, 22, 24, 27, 29, 35, 36, 37, 39, 40, 41, 42, 42, 43, 44, 45, 47, 49, 54, 55],
         6:[46, 57],
         7:[38, 48],
         8:[1],
         9:[53]
         }
new_cls = dict()
tf_keys=[i for i in range(60)]
for i in range(60):
    for key, value in zip(tf_dict.keys(), tf_dict.values()):
        if i in value:
            new_cls[i]=key

with open(os.path.join(base_path,'annotations.json'), 'r') as read_file:
    json_data = json.load(read_file)

json_data_modified = copy.deepcopy(json_data) # 원본 json data
json_data_modified['annotations']=[] # 수정하여 저장할 json data

# print(json_data.keys())
# dict_keys(['info', 'images', 'annotations', 'scene_annotations', 'licenses', 'categories', 'scene_categories'])

id_cnt = 1
for i, data in enumerate(json_data['annotations']):
    img_id = data['image_id']
    cls_id = data['category_id']
    new_cls_id = new_cls[cls_id] # 10개의 클래스로 기존 클래스 변경

    img_area = img_info[img_id]['height']*img_info[img_id]['width'] # 원본 이미지 넓이
    instance_area = data['area'] # instance 넓이

    if_save_ann = False if instance_area*100 < img_area else True # instance가 너무 작으면 제외하기 위한 flag
    if(if_save_ann):
        temp = copy.deepcopy(json_data['annotations'][i])
        json_data_modified['annotations'].append(temp)
        json_data_modified['annotations'][-1]['id']=id_cnt
        json_data_modified['annotations'][-1]['category_id']=new_cls_id
        id_cnt+=1
print(f'>>>>> modification complete!')

In [ ]:
"""
bbox 사이즈가 작은 annotation을 지우면서 image는 존재하지만, annotation이 존재하지 경우 발생
이 경우, naive faster_rcnn -> getitem에서 문제 발생
annotation이 없는 이미지 info는 삭제할 필요가 있음
삭제하면서 image id도 0부터 차례대로 다시 설정해주어야 함
"""
previous_img_id = 0
origin_img_id = []
new_img_id = []
# annotation이 없는 이미지 id를 확인하고, 0부터 이미지 id를 재생성
for i, data in enumerate(json_data_modified['annotations']):
    if data['image_id'] not in origin_img_id:
        origin_img_id.append(data['image_id'])
        new_img_id.append(previous_img_id)
        previous_img_id+=1

if len(origin_img_id)!=len(new_img_id):
    print(f'image index not match.... cannot create new image index!')

tf_img_id = dict(zip(origin_img_id, new_img_id))

# annotation에 적힌 이미지 id를 재생성한 id로 변경
for i, data in enumerate(json_data_modified['annotations']):
    json_data_modified['annotations'][i]['image_id'] =tf_img_id[data['image_id']]
    

# annotation과 일치하도록 image info의 id도 변경
json_data_modified['images']=[]
for i, data in enumerate(json_data['images']):
    if data['id'] in origin_img_id:
        temp = copy.deepcopy(json_data['images'][i])
        json_data_modified['images'].append(temp)
        json_data_modified['images'][-1]['id']=tf_img_id[data['id']]

print(f'>>>> (image id & annotation image id) modification complete!!!')

# json category 수정

In [ ]:
new_cat_dict = dict()
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
json_data_modified['categories']=[]

for i,cls_name in enumerate(classes):
    json_data_modified['categories'].append({'id':i, 'name':cls_name, 'supercategory':cls_name})

print(f'>>> category modification complete!!!')

# 수정한 json 저장

In [ ]:
with open('./new_annotations.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_data_modified, make_file, indent='\t')
print(f'>>>> output file name : new_annoations.json')